In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [2]:
##プリコネRのキャラ毎のhttpリンクを取得
main_html_name = "https://priconneredive.gamedbs.jp"
r = requests.get(main_html_name)
soup = BeautifulSoup(r.text,'lxml')

char_htmls = []
for char_a_tag in soup.find_all("a",class_=re.compile("hvr-grow")):
    char_htmls.append(char_a_tag.get('href'))

In [3]:
char_htmls ##確認

['https://priconneredive.gamedbs.jp/chara/show/104001',
 'https://priconneredive.gamedbs.jp/chara/show/110701',
 'https://priconneredive.gamedbs.jp/chara/show/100601',
 'https://priconneredive.gamedbs.jp/chara/show/103201',
 'https://priconneredive.gamedbs.jp/chara/show/102301',
 'https://priconneredive.gamedbs.jp/chara/show/108601',
 'https://priconneredive.gamedbs.jp/chara/show/105501',
 'https://priconneredive.gamedbs.jp/chara/show/113001',
 'https://priconneredive.gamedbs.jp/chara/show/106301',
 'https://priconneredive.gamedbs.jp/chara/show/109201',
 'https://priconneredive.gamedbs.jp/chara/show/100901',
 'https://priconneredive.gamedbs.jp/chara/show/101801',
 'https://priconneredive.gamedbs.jp/chara/show/110101',
 'https://priconneredive.gamedbs.jp/chara/show/106601',
 'https://priconneredive.gamedbs.jp/chara/show/104401',
 'https://priconneredive.gamedbs.jp/chara/show/111701',
 'https://priconneredive.gamedbs.jp/chara/show/112401',
 'https://priconneredive.gamedbs.jp/chara/show/1

In [4]:
##取得できるか確認
html = char_htmls[0]

r = requests.get(html)
soup = BeautifulSoup(r.text,'xml')

#プロフィール取得(練習)
profile = soup.find("h3",text="プロフィール").find_next_sibling()
profile

<ul class="shadowbox tblbox flexbox flexwrap">
<li>身長</li><li>158cm</li>
<li>体重</li><li>44kg</li>
<li>誕生日</li><li>6月6日</li>
<li>血液型</li><li>AB型</li>
<li>年齢</li><li>13歳</li>
<li>種族</li><li>エルフ</li>
<li class="to">ギルド</li><li class="to">フォレスティエ</li>
<li class="to">趣味</li><li class="to">友達作りのイメトレ</li>
<li class="to">キャッチコピー</li><li class="to">「友達」に憧れる、エルフの森の女の子</li>
<li class="to">CV</li><li class="to">花澤香菜</li>
<li class="to">現実の名前</li><li class="to">双葉 碧</li>
</ul>

In [5]:
#名前+(守備)位置を取得(練習)
name = soup.find("h2",class_=re.compile("cmnatl h2"))
type(name.text.split()[0])

str

In [6]:
##プロフィール関係の値を取得できるか 確認
profile_array = profile.find_all("li")
profile_array[1].text

'158cm'

In [7]:
##キャラの攻撃(ついでに実装されているか)の取得確認
div_tag = soup.find("h3",text="キャラの特徴").find_next_sibling().find(text=re.compile("【..】"))
div_tag[4:]

'最後衛から状態異常の矢を放つ、ぼっちの射手。'

In [8]:
##確認(結局意味なさそうだった)
feature = soup.find("h3",text="キャラの特徴").find_next_sibling()
feature.find("br").text.split("。")[0]
##inst_check[4:].split("。")[0]+"。" #ミソギ(ハロウィン用)

'ダメージと共に付与する毒や麻痺の追加効果により、防御力の高い敵であっても、確実にＨＰを削りとる'

In [9]:
catch_copies = []#既に存在するキャッチコピーかを調べる

#pdで行うと文字列(str型obj)を入れれない？ため dictで集める
char_dict = {"名前":[],
             "現実の名前":[],
             "ギルド":[],
             "位置":[],
             "攻撃":[],
             "身長":[],
             "体重":[],
             "誕生日":[],
             "血液型":[],
             "年齢":[],
             "種族":[],
             "趣味":[],
             "CV":[],
             "キャッチコピー":[],
             "特徴":[],
             "攻撃特徴":[],
             "実装bool":[],
             "デフォルトbool":[],
             "オリジナルbool":[],
             "デフォルメURL":[]}
  

##各キャラのURL上データのスクレイピング
for char_index,char_url in enumerate(char_htmls):
  
    ## urlからコード内容を読み取る
    r = requests.get(char_url)
    soup = BeautifulSoup(r.text,'lxml')
    
    ##名前+(守備)位置を取得
    name_pos_soup = soup.find("h2",class_=re.compile("cmnatl h2"))
    name_pos_array = name_pos_soup.text.split() #名前の後ろに空白があったのを利用して分割
    char_dict["名前"].append(name_pos_array[0])
    char_dict["位置"].append(name_pos_array[1])
    
    
    ##プロフィール取得
    profile = soup.find("h3",text="プロフィール").find_next_sibling()
    profile_array = profile.find_all("li")
    
    ##プロフィールに一致するlabelに次の兄弟ノードの値を入れる
    for i in range(0,len(profile_array),2):
        
         #キャッチコピーが既に存在するかのチェック(未実装)
        if(profile_array[i].text == "キャッチコピー"):
            if(profile_array[i+1] in catch_copies):#既に存在
                char_dict["デフォルトbool"].append(0)
                default_bool = 0
            else: 
                char_dict["デフォルトbool"].append(1) #原案キャラ
                catch_copies.append(profile_array[i+1])#新たなキャッチコピーとする
           
        char_dict[profile_array[i].text].append(profile_array[i+1].text)
        
      
    ##実装のチェック
    imple_check =  soup.find("h3",text="キャラの特徴").find_next_sibling().find(text=re.compile("【..】"))
    if imple_check == None: #未実装
        char_dict["実装bool"].append(0)
        char_dict["攻撃"].append('')
        char_dict["特徴"].append('')
        char_dict["攻撃特徴"].append('')
    else:
        char_dict["実装bool"].append(1)
        char_dict["攻撃"].append(imple_check[1:3])

        
        #攻撃特徴文を読み取る
        feature = soup.find("h3",text="キャラの特徴").find_next_sibling()
        if(feature.find("br") == None): ##ミソギ(ハロウィン) など?
            texts = imple_check[4:].split("。")
            char_dict["特徴"].append(texts[0]+"。 ")
                ##攻撃特徴の抽出(。が 二つ以上ある場合があるため)
            feature_text = ""
            for text in texts[1:]:
                if(text != ''):###最後の。分割を無視する
                    feature_text += text+"。"
            
            char_dict["攻撃特徴"].append(feature_text);
        else:
            char_dict["特徴"].append(imple_check[4:])
            #### print(char_dict["名前"][char_index],feature)####処理エラーが起きたため、確認+文章の取得時に使用############
                ##攻撃特徴の抽出(。が二つ以上ある時があるため)
            texts = feature.text.replace("<br/>",'').split("。")
            feature_text = ""
            for text in texts[1:]:
                if(text != ''):###最後の 。分割を無視する
                    feature_text += text+"。"
            
            char_dict["攻撃特徴"].append(feature_text)

            
    ##デフォルメ画像URLを取得(未実装キャラはデカい画像にする)
    if char_dict["実装bool"][char_index] == 1:
        defoImage_url = soup.find("a",title=re.compile("1SDイメージ"))
        char_dict["デフォルメURL"].append(defoImage_url.get("href"))
    else: ###未実装キャラの処理
        defoImage_url = soup.find("a",title=re.compile("1イメージ"))
        char_dict["デフォルメURL"].append(defoImage_url.get("href"))
    
    
    ##プリコネRのキャラかどうかを調べる
    if char_dict["実装bool"][char_index] == 0:
        char_dict["オリジナルbool"].append(1)##未実装キャラ [現実の名前の記載が無いため先に処理]
    elif char_dict["デフォルトbool"][char_index] == 0:
        char_dict["オリジナルbool"].append(1)##衣装が違う(イベント時)デフォルトキャラ ['（'がつくため先に処理]
    elif "（" in char_dict["名前"][char_index] or char_dict["名前"][char_index] == char_dict["現実の名前"][char_index] or len(char_dict["現実の名前"][char_index]) == 0:
        char_dict["オリジナルbool"].append(0)##コラボキャラ(2020/6/10 の今のところは)
    else:
        char_dict["オリジナルbool"].append(1)##デフォルトboolと同じ
    
    
    ###今回のループ内容を表示
    print("取得 No."+str(1+char_index)+" /"+str(len(char_htmls))+"    名前: "+char_dict["名前"][char_index])
 
    
    ##2秒待機
    time.sleep(2)
    
    
## 確認用の出力(念のため)
char_dict

取得 No.1 /119    名前: アオイ
取得 No.2 /119    名前: アオイ（編入生）
取得 No.3 /119    名前: アカリ
取得 No.4 /119    名前: アキノ
取得 No.5 /119    名前: アヤネ
取得 No.6 /119    名前: アヤネ（クリスマス）
取得 No.7 /119    名前: アユミ
取得 No.8 /119    名前: アユミ（ワンダー）
取得 No.9 /119    名前: アリサ
取得 No.10 /119    名前: アン
取得 No.11 /119    名前: アンナ
取得 No.12 /119    名前: イオ
取得 No.13 /119    名前: イオ（サマー）
取得 No.14 /119    名前: イノリ
取得 No.15 /119    名前: イリヤ
取得 No.16 /119    名前: イリヤ（クリスマス）
取得 No.17 /119    名前: ウヅキ（デレマス）
取得 No.18 /119    名前: エミリア
取得 No.19 /119    名前: エリコ
取得 No.20 /119    名前: エリコ（バレンタイン）
取得 No.21 /119    名前: カオリ
取得 No.22 /119    名前: カオリ（サマー）
取得 No.23 /119    名前: カスミ
取得 No.24 /119    名前: カスミ（マジカル）
取得 No.25 /119    名前: カヤ
取得 No.26 /119    名前: キャル
取得 No.27 /119    名前: キャル（サマー）
取得 No.28 /119    名前: キャル（ニューイヤー）
取得 No.29 /119    名前: キョウカ
取得 No.30 /119    名前: キョウカ（ハロウィン）
取得 No.31 /119    名前: クウカ
取得 No.32 /119    名前: クウカ（オーエド）
取得 No.33 /119    名前: クリスティーナ
取得 No.34 /119    名前: クリスティーナ（クリスマス）
取得 No.35 /119    名前: クルミ
取得 No.36 /119    名前: クルミ（クリスマス）
取得 No.3

{'名前': ['アオイ',
  'アオイ（編入生）',
  'アカリ',
  'アキノ',
  'アヤネ',
  'アヤネ（クリスマス）',
  'アユミ',
  'アユミ（ワンダー）',
  'アリサ',
  'アン',
  'アンナ',
  'イオ',
  'イオ（サマー）',
  'イノリ',
  'イリヤ',
  'イリヤ（クリスマス）',
  'ウヅキ（デレマス）',
  'エミリア',
  'エリコ',
  'エリコ（バレンタイン）',
  'カオリ',
  'カオリ（サマー）',
  'カスミ',
  'カスミ（マジカル）',
  'カヤ',
  'キャル',
  'キャル（サマー）',
  'キャル（ニューイヤー）',
  'キョウカ',
  'キョウカ（ハロウィン）',
  'クウカ',
  'クウカ（オーエド）',
  'クリスティーナ',
  'クリスティーナ（クリスマス）',
  'クルミ',
  'クルミ（クリスマス）',
  'クロエ',
  'グレア',
  'コッコロ',
  'コッコロ（サマー）',
  'コッコロ（ニューイヤー）',
  'コッコロ（プリンセス）',
  'サレン',
  'サレン（サマー）',
  'シオリ',
  'シオリ（マジカル）',
  'シズル',
  'シズル（バレンタイン）',
  'シノブ',
  'シノブ（ハロウィン）',
  'ジュン',
  'ジータ',
  'スズナ',
  'スズナ（サマー）',
  'スズメ',
  'スズメ（サマー）',
  'スズメ（ニューイヤー）',
  'タマキ',
  'タマキ（サマー）',
  'チエル',
  'チカ',
  'チカ（クリスマス）',
  'ツムギ',
  'トモ',
  'ナナカ',
  'ニノン',
  'ニノン（オーエド）',
  'ネネカ',
  'ノゾミ',
  'ノゾミ（クリスマス）',
  'ハツネ',
  'ヒヨリ',
  'ヒヨリ（ニューイヤー）',
  'ペコリーヌ',
  'ペコリーヌ（サマー）',
  'ペコリーヌ（プリンセス）',
  'マコト',
  'マコト（サマー）',
  'マツリ',
  'マヒル',
  'マヒル（レンジャー）',
  'マホ',
  'マホ（サマー）',
  'ミオ（デレマス）',


In [10]:
#今日の日付をcsv名に追加する用 (注意:文字列化しないと挿入できなかった)
import datetime
d_today = datetime.date.today()#ローカルな日付

##indexを表示しない形式でcsv化
char_df = pd.DataFrame(char_dict).set_index("名前",inplace=False)
char_df.to_csv("priconneR"+str(d_today)+".csv")